# Preprocessing 

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
sys.path.append("../src/")

In [3]:
from utilities import *

In [4]:
pd.set_option("display.max_columns", None)

### One Hot Encoding 

In [5]:
filepath = os.path.join(dir_dict["feature_engineered"], f"completed_feateng.parquet")
df = pd.read_parquet(filepath)

In [6]:
df.select_dtypes(exclude=["number", "bool"])

,event_name,event_date,event_city,event_state,event_country,fight_bout,fight_method,fight_time_format,fight_referee,fight_details,fight_weight_class,fighter_name,fighter_nickname,fighter_stance,fighter_dob,opponent_name,opponent_nickname,opponent_stance,opponent_dob
0,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,open_weight_bout,decision_unanimous,1 Rnd (18),John McCarthy,<NA>,open_weight,Dan Severn,The Beast,southpaw,1958-06-08,David Abbott,Tank,switch,NaT
1,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,ultimate_ultimate_95_tournament_title_bout,decision_unanimous,1 Rnd + OT (27-3),John McCarthy,<NA>,open_weight,Dan Severn,The Beast,southpaw,1958-06-08,Oleg Taktarov,The Russian Bear,orthodox,1967-08-26
2,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,open_weight_bout,submission,1 Rnd (15),John McCarthy,Ankle Lock From Inoki-Ali Position,open_weight,Dave Beneteau,Dangerous,orthodox,NaT,Oleg Taktarov,The Russian Bear,orthodox,1967-08-26
3,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,open_weight_bout,submission,1 Rnd (15),John McCarthy,Rear Naked Choke,open_weight,Keith Hackney,The Giant Killer,sideways,NaT,Marco Ruas,King of the Streets,orthodox,1961-01-23
4,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,ultimate_ultimate_95_tournament_title_bout,decision_unanimous,1 Rnd + OT (27-3),John McCarthy,<NA>,open_weight,Oleg Taktarov,The Russian Bear,orthodox,1967-08-26,Dan Severn,The Beast,southpaw,1958-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13475,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,middleweight_bout,decision_split,5 Rnd (5-5-5-5-5),Herb Dean,Derek Cleary ...,middleweight,Sean Strickland,<NA>,orthodox,1991-02-27,Jared Cannonier,The Killa Gorilla,switch,1984-03-16
13476,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,flyweight_bout,ko_tko,3 Rnd (5-5-5),Keith Peterson,Punch to Head At Distance,flyweight,Alessandro Costa,Nono,orthodox,1996-01-28,Amir Albazi,The Prince,orthodox,1993-10-27
13477,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,welterweight_bout,decision_unanimous,3 Rnd (5-5-5),Herb Dean,Adalaide Byrd ...,welterweight,Bryan Battle,Pooh Bear,orthodox,1994-09-21,Rinat Fakhretdinov,Gladiator,orthodox,1991-09-28
13478,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,lightweight_bout,decision_unanimous,3 Rnd (5-5-5),Mark Smith,Mike Bell ...,lightweight,Damir Ismagulov,Qazaq,orthodox,1991-02-03,Arman Tsarukyan,Ahalkalakets,orthodox,1996-10-11


In [7]:
to_drop = ["event_name", "event_date", "event_city", "event_state", "event_country",
           "fight_bout", "fight_details", "fighter_dob", "opponent_dob", 
           "fighter_name", "opponent_name", "fighter_nickname", "opponent_nickname"]

In [8]:
df = df.drop(to_drop, axis=1)

In [9]:
cat_cols = df.select_dtypes(exclude=["number", "bool"]).columns
cat_cols

Index(['fight_method', 'fight_time_format', 'fight_referee',
       'fight_weight_class', 'fighter_stance', 'opponent_stance'],
      dtype='object')

In [10]:
df[cat_cols] = df[cat_cols].astype("category")

In [11]:
df = pd.concat([df, pd.get_dummies(df[cat_cols])], axis=1)

In [12]:
df = df.drop(cat_cols, axis=1)

In [13]:
df.dtypes

fight_id                        uint64
fight_round                      uint8
fight_fighter_overall_kd       float64
fight_fighter_round1_kd        float64
fight_fighter_round2_kd        float64
                                ...   
opponent_stance_open_stance      uint8
opponent_stance_orthodox         uint8
opponent_stance_sideways         uint8
opponent_stance_southpaw         uint8
opponent_stance_switch           uint8
Length: 1382, dtype: object

In [15]:
bool_cols = df.select_dtypes("bool").columns
df[bool_cols] = df[bool_cols].astype("uint8")

In [19]:
df = df.drop("fight_opponent_win", axis=1)

In [20]:
df

fight_id  fight_round  fight_fighter_overall_kd  \
0        974788696932076864            1                       0.0   
1       5929914313661320301            2                       0.0   
2      15303123771856675855            1                       0.0   
3       4483802662300382653            1                       0.0   
4       5929914313661320301            2                       0.0   
...                     ...          ...                       ...   
13475   5666808103719092317            5                       0.0   
13476  15663215643086211781            3                       0.0   
13477  10304476116225125602            3                       0.0   
13478  14877435572882344614            3                       0.0   
13479  17277582278479194313            3                       0.0   

       fight_fighter_round1_kd  fight_fighter_round2_kd  \
0                          0.0                      NaN   
1                          0.0                      0.0   
2                          0.0                      NaN   
3                          0.0                      NaN   
4                          0.0                      0.0   
...                        ...                      ...   
13475                      0.0                      0.0   
13476                      0.0                      0.0   
13477                      0.0                      0.0   
13478                      0.0                      0.0   
13479                      0.0                      0.0   

       fight_fighter_overall_sig_str_%  fight_fighter_round1_sig_str_%  \
0                                 0.87                            0.87   
1                                 0.30                            0.37   
2                                 0.40                            0.40   
3                                 0.25                            0.25   
4                                 0.27                            0.41   
...                                ...                             ...   
13475                             0.38                            0.27   
13476                             0.23                            0.26   
13477                             0.18                            0.33   
13478                             0.41                            0.50   
13479                             0.50                            0.55   

       fight_fighter_round2_sig_str_%  fight_fighter_overall_td_%  \
0                                 NaN                        0.50   
1                                0.09                         NaN   
2                                 NaN                         NaN   
3                                 NaN                         NaN   
4                                0.08                        0.00   
...                               ...                         ...   
13475                            0.38                        1.00   
13476                            0.11                         NaN   
13477                            0.20                        0.00   
13478                            0.39                         NaN   
13479                            0.36                        0.54   

       fight_fighter_round1_td_%  fight_fighter_round2_td_%  \
0                            0.5                        NaN   
1                            NaN                        NaN   
2                            NaN                        NaN   
3                            NaN                        NaN   
4                            0.0                        NaN   
...                          ...                        ...   
13475                        1.0                        NaN   
13476                        NaN                        NaN   
13477                        NaN                        NaN   
13478                        NaN                        NaN   
13479                        0.0                       0.7

In [23]:
df = df.fillna(df.mean())

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [29]:
def model_performance(df):
    df_num = df.select_dtypes(["number", "bool"])
    bool_cols = df_num.select_dtypes("bool").columns
    df_num[bool_cols] = df_num[bool_cols].astype("uint8")
    df_num = df_num.fillna(0)

    to_drop = [col for col in df_num.columns if "fight_" in col] + ["fighter_id", "opponent_id"]
    to_drop.remove("fight_fighter_win")
    df_num = df_num.drop(to_drop, axis=1)

    df_num["fight_winner"] = df_num["fight_fighter_win"]
    df_num = df_num.drop(["fight_fighter_win"], axis=1)

    X, y = df_num.drop("fight_winner", axis=1), df_num["fight_winner"]

    return np.mean(cross_val_score(LogisticRegression(random_state=42), X, y, cv=5, n_jobs=cpu_count()))

In [30]:
model_performance(df)

/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

0.8433234421364985